# Initial Configs


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [ ]:
from __future__ import division
#import libraries
# from datetime import datetime, timedelta, date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import swifter
import seaborn as sns

#do not show warnings
import warnings
warnings.filterwarnings("ignore")

#import plotly for visualization
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
import sys
from IPython.core.display import display, HTML
sys.path.append('..')
pyoff.init_notebook_mode()

from pycaret.classification import *
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report, \
precision_score, recall_score, f1_score, fbeta_score, accuracy_score, matthews_corrcoef, precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *

from tqdm import tqdm_notebook as tqdm

# Package configs

In [ ]:
pd.set_option('display.max_columns', None)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('plotting.backend', 'pandas_bokeh')
bokeh.io.output_notebook(INLINE)

# Functions

In [ ]:
# Print rapido para dimensão do Dataframe
def SZ(df):
    print(f"""
--- Dimensão ---
Linhas:  {df.shape[0]}
Colunas: {df.shape[1]}""")

In [ ]:
# Print rapido para dTypes do Dataframe
def DT(df):
    print(f"""
--- DataTypes ---
{df.dtypes}
    """)

In [ ]:
#print de distribuição de features numéricas
def distFeaturesNumericas(df,num_columns):
    for col in num_columns:
        plt.figure(figsize = (30,5))
        plt.hist([df[df['contem_vitima_fatal'] == 0][col],df[df['contem_vitima_fatal'] == 1][col]],bins = 30, color=['blue','pink'], label=['no fatal','yes fatal'])
        plt.legend()
        plt.ylabel('Count')
        plt.title(col)
        plt.show()

In [ ]:
#print de distribuição de features categóricas
def distFeaturesCategoricas(df,cat_columns):
    for col in cat_columns:
        N_0 = len(df[df['contem_vitima_fatal'] == 0][col].value_counts().index)
        count_0 = df[df['contem_vitima_fatal'] == 0][col].value_counts().values

        ind_0 = np.arange(N_0)  # the x locations for the groups
        width = 0.35       # the width of the bars

        fig = plt.figure(figsize = (30,5))
        ax = fig.add_subplot(111)
        rects1 = ax.bar(ind_0, count_0, width, color='blue')

        N_1 = len(df[df['contem_vitima_fatal'] == 1][col].value_counts().index)
        ind_1 = np.arange(N_1)  # the x locations for the groups
        count_1 = df[df['contem_vitima_fatal'] == 1][col].value_counts().values
        rects2 = ax.bar(ind_1+width, count_1, width, color='pink')

        ax.set_ylabel('Count')
        ax.set_title(col)
        ax.set_xticks(ind_0 + width / 2)
        ax.set_xticklabels(df[df['contem_vitima_fatal'] == 0][col].value_counts().index)

        ax.legend( (rects1[0], rects2[0]), ('not fatal', 'yes fatal') )

        plt.show()

In [ ]:
df_train = pd.read_parquet('../data/processed/dataset_SMOTENC_TREINO_v1.parquet')
df_test = pd.read_parquet('../data/processed/dataset_SMOTENC_TESTE_v1.parquet')

# Data

## Loads

In [ ]:
cluster = 0

In [ ]:
df_train = df_train.loc[df_train['cluster_coords']==cluster,:]
df_test = df_test.loc[df_test['cluster_coords']==cluster,:]

In [ ]:
SZ(df_train)
DT(df_train)

Carregando dados de validação/avaliação

In [ ]:
df_2020 = pd.read_parquet('../data/processed/datasetSMOTENC2020_v1.parquet')
df_2020 = df_2020.loc[df_2020['cluster_coords']==cluster,:]
df_2020 = df_2020.drop(['data_inversa'], axis=1)

SZ(df_2020)
DT(df_2020)

# Modelling Classification via PyCaret

You can see the documentation in [Pycaret Classification](https://pycaret.org/classification/)

In [ ]:
ignore_columns = ['id', 'cluster_coords']

In [ ]:
cat_columns = [
      'dia_semana'
    , 'uf'
    , 'fase_dia'
    , 'sentido_via'
    , 'condicao_metereologica'
    , 'tipo_pista'
    , 'tracado_via'
    , 'uso_solo'
    , 'em_janela_feriado'
    #, 'cluster_coords'
]

In [ ]:
num_columns = [
    'risco'
    , 'risco_morte'
    , 'pessoas'
    , 'coordenada_x'
    , 'coordenada_y'
    , 'coordenada_z'
]

In [ ]:
#distFeaturesNumericas(df_train,num_columns)

In [ ]:
distFeaturesCategoricas(df_test,cat_columns)

In [ ]:
folds = 5

In [ ]:
exp_reg = setup(data=df_train,
                test_data=df_test,
                target = 'contem_vitima_fatal',
                numeric_features = num_columns,
                categorical_features = cat_columns,
                ignore_features = ignore_columns,
                normalize=True,
                pca=False,
                create_clusters=False,
                fix_imbalance=False,
                #fix_imbalance_method = smote_nc,
                data_split_stratify=True,
                ignore_low_variance=True,
                transformation=False,
                train_size=0.8,
                combine_rare_levels=True,
                fold=folds,
                rare_level_threshold=0.10,
                feature_ratio=False,
                feature_interaction=False,
                feature_selection=True,
                feature_selection_method='boruta',
                remove_multicollinearity=True,
                remove_perfect_collinearity=True,
                remove_outliers=False,
                polynomial_features=False,
                session_id=123,
#                 log_experiment=True,
#                 experiment_name='Predict Fatal Victim smotek5',
#                 log_plots=True,
#                 log_profile=False,
#                 log_data=True,
                silent=True,
                verbose=True,
                profile=False,
               )

In [ ]:
models()

## Comparing All Models

Comparing all models to evaluate performance is the recommended starting point for modeling once the setup is completed (unless you exactly know what kind of model you need, which is often not the case). This function trains all models in the model library and scores them using stratified cross validation for metric evaluation. The output prints a score grid that shows average Accuracy, Recall, Precision, F1, Kappa, and MCC accross the folds (10 by default) along with training times.

In [ ]:
%%time
best = compare_models(
    sort='AUC',
    exclude=['knn', 'ridge', 'svm', 'lr', 'ada', 'lda', 'nb', 'qda'],
    fold=folds,
    n_select=3,
    turbo=True)

Note: The AUC metric is not available for Multiclass classification however the column will still be shown with zero values to maintain consistency between the Binary Classification and Multiclass Classification display grids.

## Create a Model

`create_model` is the most granular function in PyCaret and is often the foundation behind most of the PyCaret functionalities. As the name suggests this function trains and evaluates a model using cross validation that can be set with fold parameter. The output prints a score grid that shows Accuracy, Recall, Precision, F1, Kappa and MCC by fold.

In [ ]:
model_cat = create_model('catboost', fold=folds)

In [ ]:
model_lightgbm = create_model('lightgbm', fold=folds)

## Tune a Model

When a model is created using the create_model() function it uses the default hyperparameters to train the model. In order to tune hyperparameters, the tune_model() function is used. This function automatically tunes the hyperparameters of a model using Random Grid Search on a pre-defined search space. The output prints a score grid that shows Accuracy, AUC, Recall, Precision, F1, Kappa, and MCC by fold for the best model. To use the custom search grid, you can pass custom_grid parameter in the tune_model function (see 9.2 KNN tuning below).

In [ ]:
param_test ={'learning_rate' : [1e-5, 1e-3, 1e-2, 5e-1, 1e-1],
             'n_estimators' : sp_randint(10, 1000),
             'num_leaves': sp_randint(6, 75), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

tuned_model_lgbm = tune_model(model_lgbm, n_iter=100, custom_grid=param_test, optimize='AUC', choose_better=True)

The `tune_model()` function is a random grid search of hyperparameters over a pre-defined search space.

In [ ]:
tuned_model_cat = tune_model(model_cat, n_iter=100, optimize='AUC', choose_better=True)

## Ensemble Model

In [ ]:
bagging_tuned_model_cat = ensemble_model(tuned_model_cat, fold=folds)

In [ ]:
bagging_tuned_model_lgbm = ensemble_model(tuned_model_lgbm, fold=folds)

## Stack Model

In [ ]:
stacked_models = stack_models(estimator_list = best[1:], meta_model = best[0])

## Plot a Model

Before model finalization, the `plot_model()` function can be used to analyze the performance across different aspects such as AUC, confusion_matrix, decision boundary etc. This function takes a trained model object and returns a plot based on the test / hold-out set.

In [ ]:
plot_model(model_cat, 'confusion_matrix')

In [ ]:
plot_model(model_cat, 'class_report')

In [ ]:
plot_model(model_cat, 'feature')

In [ ]:
plot_model(model_lgbm, 'auc')

In [ ]:
plot_model(model_lgbm, 'pr')

In [ ]:
plot_model(model_lgbm, 'error')

In [ ]:
plot_model(model_lgbm, 'calibration')

In [ ]:
calibrated_lgbm = calibrate_model(model_lgbm, method='isotonic')

In [ ]:
plot_model(calibrated_lgbm, 'calibration')

In [ ]:
plot_model(model_lgbm, 'threshold')

## Calibrate Model

In [ ]:
plot_model(model_cat, 'calibration')

In [ ]:
pred_lightgbm = predict_model(model_cat)

In [ ]:
calibrated_model = calibrate_model(model_cat, fold=folds, method='isotonic')

In [ ]:
plot_model(calibrated_model, 'calibration')

In [ ]:
pred_lightgbm = predict_model(calibrated_model)

In [ ]:
calibrated_model2 = calibrate_model(model_cat, fold=folds, method='sigmoid')

In [ ]:
plot_model(calibrated_model2, 'calibration')

In [ ]:
pred_lightgbm = predict_model(calibrated_model2)

In [ ]:
plot_model(model_lightgbm, 'calibration')

In [ ]:
predict_model(model_lightgbm)

In [ ]:
calibrated_lightgbm = calibrate_model(model_lightgbm, fold=folds, method='isotonic')

In [ ]:
plot_model(calibrated_lightgbm, 'calibration')

In [ ]:
predict_model(calibrated_lightgbm)

In [ ]:
calibrated_lightgbm2 = calibrate_model(model_lightgbm, fold=folds, method='sigmoid')

In [ ]:
plot_model(calibrated_lightgbm2, 'calibration')

In [ ]:
predict_model(calibrated_lightgbm2)

## Optimize Threshold

In [ ]:
from itertools import product # Biblioteca para achar valores mais próximos entre 2 arrays

In [ ]:
df_predicted = predict_model(model_cat, probability_threshold=0.0)

In [ ]:
f1_score_array, precision_score_array, recall_score_array = [], [], []
thresholds_list = np.linspace(0,1,101)

with tqdm(total = len(thresholds_list)) as pbar:
    for threshold in thresholds_list:
        df_predicted['contem_vitima_fatal'] = df_predicted['contem_vitima_fatal'].astype(int)
        df_predicted['y_pred'] = df_predicted.apply(lambda x: 1 if x['Score'] >= threshold else 0, axis=1)

        f1_score_array.append(f1_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
#         precision_score_array.append(precision_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
#         recall_score_array.append(recall_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
        
        pbar.update()
    
    best_threshold_f1 = thresholds_list[np.argmax(f1_score_array)]
    plt.plot(thresholds_list, f1_score_array)
    plt.axvline(x=best_threshold_f1)
    plt.title(f'Best threshold based on F1-Score = {best_threshold_f1}')
    plt.show()
    
#     #Find closest values between precision and recall arrays
#     closest_pr = sorted(product(precision_score_array, recall_score_array), key=lambda t: abs(t[0]-t[1]))[0]
    
#     # Get index of closest values between precision and recall arrays
#     indices_precision = [i for i, x in enumerate(precision_score_array) if x == closest_pr[0]]
#     indices_recall = [i for i, x in enumerate(precision_score_array) if x == closest_pr[1]]
#     indice_best_threshold = list(set(indices_precision) & set(indices_recall))[0]
    
#     plt.plot(thresholds_list, precision_score_array)
#     plt.plot(thresholds_list, recall_score_array)
#     plt.axvline(x=thresholds_list[indice_best_threshold])
#     plt.title(f'Best threshold based on Precision x Recall = {thresholds_list[indice_best_threshold]}')
#     plt.show()

In [ ]:
df_predicted['y_pred'] = df_predicted.apply(lambda x: 1 if x['Score'] >= best_threshold_f1 else 0, axis=1).astype(int)
df_predicted['contem_vitima_fatal'] = df_predicted['contem_vitima_fatal'].astype(int)

In [ ]:
print('AUC:', roc_auc_score(df_predicted['contem_vitima_fatal'], df_predicted['Score']))
print('Accuracy:', accuracy_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
print('Precision:', precision_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
print('Recall:', recall_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
print('F1-Score:', f1_score(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
print(classification_report(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))
print(confusion_matrix(df_predicted['contem_vitima_fatal'], df_predicted['y_pred']))

In [ ]:
df_predicted_2020 = predict_model(model_cat, probability_threshold=0.0, data=df_2020)

In [ ]:
df_predicted_2020['y_pred'] = df_predicted_2020.apply(lambda x: 1 if x['Score'] >= best_threshold_f1 else 0, axis=1).astype(int)
df_predicted_2020['contem_vitima_fatal'] = df_predicted_2020['contem_vitima_fatal'].astype(int)

In [ ]:
print('AUC:', roc_auc_score(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['Score']))
print('Accuracy:', accuracy_score(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))
print('Precision:', precision_score(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))
print('Recall:', recall_score(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))
print('F1-Score:', f1_score(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))
print(classification_report(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))
print(confusion_matrix(df_predicted_2020['contem_vitima_fatal'], df_predicted_2020['y_pred']))

## Predict Model

In [ ]:
def generate_metrics(df, y_true, y_pred, average='macro', labels = [0, 1]):
    df['y_true'] = df[y_true].astype(str)
    df['y_pred'] = df[y_pred].astype(str)
    
    print(classification_report(df['y_true'], df['y_pred'], labels=labels))

In [ ]:
# tuned_model_xgb_final = finalize_model(tuned_model_xgb)

In [ ]:
df_train = get_config("X_train").reset_index()[['index']].merge(new_df.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')
del df_train['index']

In [ ]:
model_predicted_train = predict_model(model_rf, data=df_train)

In [ ]:
model_predicted_test = predict_model(model_rf)

In [ ]:
print('------------------- Train Metrics -------------------')
generate_metrics(model_predicted_train, 'contem_vitima_fatal', 'Label')

print('------------------- Test Metrics -------------------')
generate_metrics(model_predicted_test, 'contem_vitima_fatal', 'Label')

In [ ]:
model_predicted_train = predict_model(model_lgbm, data=df_train)

In [ ]:
model_predicted_test = predict_model(model_lgbm)

In [ ]:
print('------------------- Train Metrics -------------------')
generate_metrics(model_predicted_train, 'contem_vitima_fatal', 'Label')

print('------------------- Test Metrics -------------------')
generate_metrics(model_predicted_test, 'contem_vitima_fatal', 'Label')

In [ ]:
model_predicted_train = predict_model(model_cat, data=df_train, probability_threshold=0.0)

In [ ]:
model_predicted_test = predict_model(model_cat)

In [ ]:
print('------------------- Train Metrics -------------------')
generate_metrics(model_predicted_train, 'contem_vitima_fatal', 'Label')

print('------------------- Test Metrics -------------------')
generate_metrics(model_predicted_test, 'contem_vitima_fatal', 'Label')

In [ ]:
plot_data = [
    go.Histogram(
        x = tx_data['TARGET'],
        orientation='v',
        name='Segmentos'
    )
]

plot_layout = go.Layout(
        width=700,
        height=300
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
r = evaluate_all(lgbm_final_pred_test['y_true'], lgbm_final_pred_test['y_pred'])

In [ ]:
lgbm_final_pred_test.to_csv("../data/processed/Predict Revenue Purchase_LGBM_129.csv", index=False)

Finalize and Save Model

In [ ]:
model_rf_finalized = finalize_model(model_rf)

In [ ]:
save_model(model_rf, "../models/model_rf_02_02_2021", verbose=True)

In [ ]:
model_predicted_et = predict_model(model_et)

print('------------------- et -------------------')
generate_metrics(model_predicted_et, 'contem_vitima_fatal', 'Label')

In [ ]:
df_2020 = pd.read_parquet('../data/processed/datasetSMOTENC2020.parquet')
df_2020['data_inversa'] = pd.to_datetime(df_2020['data_inversa'])
df_2020.head()
# df_2020 = df_2020.drop(['data_inversa','risco_morte','em_janela_feriado'], axis=1)

SZ(new_df_resampled)
DT(new_df_resampled)
SZ(df_2020)
DT(df_2020)

In [ ]:
# df_train = get_config("X_train").reset_index()[['index']].merge(new_df_resampled.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')
# df_train.head()

In [ ]:
z_score2020 = stats.zscore(df_2020.drop(['id'], axis=1).drop(['contem_vitima_fatal'], axis=1).select_dtypes(include=[np.number]))

In [ ]:
abs_z_scores2020 = np.abs(z_score2020)
filtered_entries2020 = (abs_z_scores2020 < 3).all(axis=1)
sum(filtered_entries2020)#
new_df2020 = df_2020[filtered_entries2020]

In [ ]:
#df_2020['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');
df_2020_0 = df_2020[df_2020['contem_vitima_fatal'] == 0]
df_2020_1 = df_2020[df_2020['contem_vitima_fatal'] == 1]
print(df_2020_0.shape[0]/(df_2020_0.shape[0]+df_2020_1.shape[0]))
print(df_2020_1.shape[0]/(df_2020_0.shape[0]+df_2020_1.shape[0]))

In [ ]:
model_predicted_cat = predict_model(model_cat, data=df_2020)

In [ ]:
print('------------------- cat -------------------')
from sklearn.metrics import confusion_matrix
confusion_matrix(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label'])

In [ ]:
new_df_2020_0 = new_df2020[new_df2020['contem_vitima_fatal'] == 0]
new_df_2020_1 = new_df2020[new_df2020['contem_vitima_fatal'] == 1]
print(new_df_2020_0.shape[0]/(new_df_2020_0.shape[0]+new_df_2020_1.shape[0]))
print(new_df_2020_1.shape[0]/(new_df_2020_0.shape[0]+new_df_2020_1.shape[0]))

In [ ]:
model_predicted_cat2 = predict_model(model_cat, data=new_df2020)

In [ ]:
print('------------------- cat 2-------------------')
confusion_matrix(model_predicted_cat2['contem_vitima_fatal'], model_predicted_cat2['Label'])

In [ ]:
model_predicted_dt = predict_model(model_dt, data=df_2020)

print('------------------- dt -------------------')
confusion_matrix(model_predicted_dt['contem_vitima_fatal'], model_predicted_dt['Label'])

In [ ]:
df_train = get_config("X_train").reset_index()[['index']].merge(new_df_resampled.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')
df_test = get_config("X_test").reset_index()[['index']].merge(new_df_resampled.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')

In [ ]:
model_predicted_cat_train = predict_model(model_cat, data=df_train)
confusion_matrix(model_predicted_cat_train['contem_vitima_fatal'], model_predicted_cat_train['Label'])

In [ ]:
model_predicted_cat_test = predict_model(model_cat, data=new_df_split_test)
confusion_matrix(model_predicted_cat_test['contem_vitima_fatal'], model_predicted_cat_test['Label'])

In [ ]:
model_predicted_cat = predict_model(model_cat, data=df_2020)
confusion_matrix(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label'])

In [ ]:
model_predicted_tunedcat = predict_model(tuned_model_cat, data=df_2020)
confusion_matrix(model_predicted_tunedcat['contem_vitima_fatal'], model_predicted_tunedcat['Label'])

In [ ]:
model_predicted_tunedcat_test = predict_model(tuned_model_cat, data=new_df_split_test)
confusion_matrix(model_predicted_tunedcat_test['contem_vitima_fatal'], model_predicted_tunedcat_test['Label'])

In [ ]:
model_predicted_dt_train = predict_model(model_dt, data=df_train)
confusion_matrix(model_predicted_dt_train['contem_vitima_fatal'], model_predicted_dt_train['Label'])

In [ ]:
from sklearn.metrics import confusion_matrix
model_predicted_dt_test = predict_model(model_dt, data=new_df_split_test)
confusion_matrix(model_predicted_dt_test['contem_vitima_fatal'], model_predicted_dt_test['Label'])

In [ ]:
model_predicted_dt = predict_model(model_dt, data=df_2020)
confusion_matrix(model_predicted_dt['contem_vitima_fatal'], model_predicted_dt['Label'])

In [ ]:
print('------------------- dt test-------------------')
print("Accuracy : ",accuracy_score(model_predicted_dt_test['contem_vitima_fatal'], model_predicted_dt_test['Label']))
#print()
print("Recall : ",recall_score(model_predicted_dt_test['contem_vitima_fatal'], model_predicted_dt_test['Label']))
print("Precision : ",precision_score(model_predicted_dt_test['contem_vitima_fatal'], model_predicted_dt_test['Label']))


print('------------------- cat test-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat_test['contem_vitima_fatal'], model_predicted_cat_test['Label']))
print("Recall : ",recall_score(model_predicted_cat_test['contem_vitima_fatal'], model_predicted_cat_test['Label']))
print("Precision : ",precision_score(model_predicted_cat_test['contem_vitima_fatal'], model_predicted_cat_test['Label']))


print('------------------- dt 2020-------------------')
print("Accuracy : ",accuracy_score(model_predicted_dt['contem_vitima_fatal'], model_predicted_dt['Label']))
print("Recall : ",recall_score(model_predicted_dt['contem_vitima_fatal'], model_predicted_dt['Label']))
print("Precision : ",precision_score(model_predicted_dt['contem_vitima_fatal'], model_predicted_dt['Label']))


print('------------------- cat 2020-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Recall : ",recall_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Precision : ",precision_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))


In [ ]:
print('------------------- cat 2020-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Recall : ",recall_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Precision : ",precision_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))

In [ ]:
df_train_split = get_config("X_train").reset_index()[['index']].merge(new_df_resampled.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')
model_predicted_cat_train_split = predict_model(model_cat, data=df_train_split)
print('------------------- cat train split-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat_train_split['contem_vitima_fatal'], model_predicted_cat_train_split['Label']))
print("Recall : ",recall_score(model_predicted_cat_train_split['contem_vitima_fatal'], model_predicted_cat_train_split['Label']))
print("Precision : ",precision_score(model_predicted_cat_train_split['contem_vitima_fatal'], model_predicted_cat_train_split['Label']))

model_predicted_cat_split = predict_model(model_cat, data=new_df_split_test)
print('------------------- cat test split-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat_split['contem_vitima_fatal'], model_predicted_cat_split['Label']))
print("Recall : ",recall_score(model_predicted_cat_split['contem_vitima_fatal'], model_predicted_cat_split['Label']))
print("Precision : ",precision_score(model_predicted_cat_split['contem_vitima_fatal'], model_predicted_cat_split['Label']))


model_predicted_cat = predict_model(model_cat, data=df_2020)
print('------------------- cat 2020-------------------')
print("Accuracy : ",accuracy_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Recall : ",recall_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))
print("Precision : ",precision_score(model_predicted_cat['contem_vitima_fatal'], model_predicted_cat['Label']))

In [ ]:
df_test = get_config("X_test").reset_index()[['index']].merge(new_df_split_test.drop(ignore_columns, axis=1).reset_index(), how='left', on='index')
df_test[df_test['index']==26]

In [ ]:
new_df_split_test.sort_index().iloc[[5]]

In [ ]:
df_test.equals(new_df_split_test)